# Temporal  Tables
## Creación

In [ ]:
CREATE TABLE dbo.Temporal
(
  TemporalID INT NOT NULL IDENTITY(1,1) PRIMARY KEY
, FirstName VARCHAR(100) NOT NULL
, LastName VARCHAR(100) NOT NULL
, StartTime DATETIME2(7) GENERATED ALWAYS AS ROW START HIDDEN 
, EndTime DATETIME2(7) GENERATED ALWAYS AS ROW END HIDDEN
, PERIOD FOR SYSTEM_TIME ([StartTime], [EndTime])
)
WITH
(
SYSTEM_VERSIONING = ON ( HISTORY_TABLE = dbo.TemporalHistory )
)


## Insertar Datos

In [ ]:
INSERT INTO dbo.Temporal
    (
    FirstName, LastName
    )
SELECT Distinct FirstName, LastName
FROM person.Person 


## Verificar Datos

In [ ]:
SELECT * FROM dbo.Temporal

### Incluir Columnas de Version

In [ ]:
SELECT *, StartTime, EndTime FROM dbo.Temporal

## Modificar Datos

In [ ]:
UPDATE dbo.Temporal
set FirstName = 'Bob', LastName = 'May'
where TemporalID = 33

### Verificar Cambio en tabla corriente

In [ ]:
SELECT * , StartTime, EndTime FROM dbo.Temporal
where TemporalID = 33

### Verificar cambios en tabla historica

In [ ]:
SELECT * FROM dbo.TemporalHistory
where TemporalID = 33

In [ ]:
SELECT TemporalID, FirstName, LastName, StartTime, EndTime FROM dbo.Temporal
FOR SYSTEM_TIME ALL
where TemporalID = 33

### Cambiemos el nombre una vez mas

In [ ]:
UPDATE dbo.Temporal
set FirstName = 'John', LastName = 'Smith'
where TemporalID = 33

### Visualizemos de otra manera

In [ ]:
SELECT TemporalID, FirstName, LastName, StartTime, EndTime,
Case EndTime
when '9999-12-31 23:59:59.9999999' then 'CURRENT'
ELSE 'OUTDATED' END AS RowType
FROM dbo.Temporal
FOR SYSTEM_TIME ALL
where TemporalID = 33
order by StartTime desc

## Borrando Datos

In [ ]:
Delete from dbo.Temporal where TemporalID = 33

## Reinsertando Datos

In [ ]:
SET IDENTITY_INSERT dbo.Temporal ON

INSERT INTO dbo.Temporal
(TemporalID, FirstName ,LastName)
select Top (1) TemporalID, FirstName, LastName from TemporalHistory
ORDER by StartTime desc

SET IDENTITY_INSERT dbo.Temporal OFF

In [ ]:
SELECT TemporalID, FirstName, LastName, StartTime, EndTime,
Case EndTime
when '9999-12-31 23:59:59.9999999' then 'CURRENT'
ELSE 'OUTDATED' END AS RowType
FROM dbo.Temporal
FOR SYSTEM_TIME ALL
where TemporalID = 33
order by StartTime desc

## System_Time

### Between

In [ ]:
SELECT TemporalID, FirstName, LastName, StartTime, EndTime,
Case EndTime
when '9999-12-31 23:59:59.9999999' then 'CURRENT'
ELSE 'OUTDATED' END AS RowType
FROM dbo.Temporal
FOR SYSTEM_TIME BETWEEN '2020-06-10 20:19' and '2020-06-10 20:24'
where TemporalID = 33
order by StartTime desc

### AS OF

In [ ]:
SELECT TemporalID, FirstName, LastName, StartTime, EndTime,
Case EndTime
when '9999-12-31 23:59:59.9999999' then 'CURRENT'
ELSE 'OUTDATED' END AS RowType
FROM dbo.Temporal
FOR SYSTEM_TIME AS OF '2020-06-10 20:19' 
where TemporalID = 33
order by StartTime desc

# Modificando la Tabla Principal

## Agregando una Columna

In [ ]:
ALTER TABLE dbo.Temporal
add StrongID UNIQUEIDENTIFIER DEFAULT(NewID())

In [ ]:
select * from dbo.Temporal

In [ ]:
Update dbo.Temporal set StrongID = NEWID()
where StrongID is null

In [15]:
SELECT TemporalID, FirstName, LastName, StartTime, EndTime, StrongID,
Case EndTime
when '9999-12-31 23:59:59.9999999' then 'CURRENT'
ELSE 'OUTDATED' END AS RowType
FROM dbo.Temporal
FOR SYSTEM_TIME ALL
where TemporalID = 33
order by StartTime desc

(5 rows affected)

Total execution time: 00:00:00.008

TemporalID,FirstName,LastName,StartTime,EndTime,StrongID,RowType
33,John,Smith,2020-06-10 20:45:06.3309304,9999-12-31 23:59:59.9999999,a19af68d-8220-4370-aa27-56977c032404,CURRENT
33,John,Smith,2020-06-10 20:27:16.0448681,2020-06-10 20:45:06.3309304,NULL,OUTDATED
33,John,Smith,2020-06-10 20:19:10.0530126,2020-06-10 20:24:55.2465457,NULL,OUTDATED
33,Bob,May,2020-06-10 20:12:08.3419059,2020-06-10 20:19:10.0530126,NULL,OUTDATED
33,Evan,Adams,2020-06-10 20:10:26.3010367,2020-06-10 20:12:08.3419059,NULL,OUTDATED


## Agregando una columna NOT-NULL

### Veamos un intento fallido 

In [ ]:
ALTER TABLE dbo.Temporal
ADD [TimeStamp] RowVersion NOT NULL

In [12]:
ALTER TABLE dbo.Temporal
SET (SYSTEM_VERSIONING = OFF)

ALTER TABLE dbo.TemporalHistory
ADD [TimeStamp] RowVersion NOT NULL

ALTER TABLE dbo.Temporal
ADD [TimeStamp] RowVersion NOT NULL

ALTER TABLE dbo.Temporal
SET (SYSTEM_VERSIONING = ON ( HISTORY_TABLE = dbo.TemporalHistory, DATA_CONSISTENCY_CHECK = ON ))

Commands completed successfully.

Total execution time: 00:00:00.223

In [19]:
SELECT TemporalID, FirstName, LastName, StartTime, EndTime, StrongID, [TimeStamp],
Case EndTime
when '9999-12-31 23:59:59.9999999' then 'CURRENT'
ELSE 'OUTDATED' END AS RowType
FROM dbo.Temporal
FOR SYSTEM_TIME ALL
where TemporalID = 33
order by StartTime desc

(1 row affected)

Total execution time: 00:00:00.006

TemporalID,FirstName,LastName,StartTime,EndTime,StrongID,TimeStamp,RowType
33,John,Smith,2020-06-10 20:45:06.3309304,9999-12-31 23:59:59.9999999,a19af68d-8220-4370-aa27-56977c032404,0x00000000000073BB,CURRENT


# LIMPIANDO EL HISTORIAL

In [18]:
ALTER TABLE dbo.Temporal
SET (SYSTEM_VERSIONING = OFF)

TRUNCATE TABLE dbo.TemporalHistory

ALTER TABLE dbo.Temporal
SET (SYSTEM_VERSIONING = ON ( HISTORY_TABLE = dbo.TemporalHistory, DATA_CONSISTENCY_CHECK = ON ))

Commands completed successfully.

Total execution time: 00:00:00.057

# DATA_CONSISTENCY_CHECK = ON 

Verifica que la tabla de historial propuesta cumpla con lo siguiente:
- No Identidades
- No Indices independientes
- No Constraints
- Tabla Principal tenga una llave primaria pero la historial no. 
- CDC no este encendido en la tabla historica
- No llaves foraneas


In [22]:
exec TableToTemporal 'HumanResources.Departments'

PROVIDED TABLE NOT FOOUND

Total execution time: 00:00:00.002

# RESET - DO NOT RUN

In [ ]:
alter TABLE dbo.Temporal
set (system_versioning = off)
DROP TABLE dbo.Temporal
DROP TABLE dbo.TemporalHistory